In [3]:
from elasticsearch import Elasticsearch
import json
import openai
from openai import OpenAI
import os
from tqdm.auto import tqdm

/workspaces/llm-zoomcamp-dm/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [5]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [6]:
es_client = Elasticsearch("http://127.0.0.1:9200", api_key="8.17.6")

In [ ]:
# with open('documents.json', 'rt') as f_in:
#     documents_raw = json.load(f_in)

In [ ]:
# documents = []

# for course_dict in documents_raw:
#     for doc in course_dict['documents']:
#         doc['course'] = course_dict['course']
#         documents.append(doc)

In [7]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [13]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = 'course_questions'

# # we created the index with the settings above
# es_client.indices.create(
#     index=index_name,
#     body=index_settings
# )

# # now we are indexing the documents using the created indices
# for doc in tqdm(documents):
#     es_client.index(
#         index=index_name,
#         document=doc
#     )

In [22]:
query = "how do i run kafka?"

def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    search_results = es_client.search(index=index_name, body=search_query)

    # print(search_results)

    results=[]

    for i in search_results['hits']['hits']:
        results.append(i['_source'])

    return results

In [31]:
def build_prompt(results, query):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: {context}
""".strip()
    context = ""
    for i in results:
        context += f"Q:{i['question']}\n A:{i['text']}\n\n".strip()

    return prompt_template.format(question=query, context=context)

In [10]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{'role':'user','content':prompt}]
    )
    return response.choices[0].message.content.strip()

In [11]:
def rag(query):
    results = elastic_search(query)
    prompt = build_prompt(results,query)
    response = llm(prompt)
    return response

In [24]:
hw3 = elastic_search("How do copy a file to a Docker container?")

In [32]:
x = build_prompt(hw3, "How do copy a file to a Docker container?")

In [35]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

In [37]:
len(encoding.encode(x))

319